<a href="https://colab.research.google.com/github/SufyAD/Agentic-AI/blob/main/CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
from google.colab import userdata

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [ ]:
!pip install -qU crewai crewai-tools

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, crew
from crewai_tools import SerperDevTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

### Important Steps to create a CrewAI based Project
1.   Create a simple `augmented_llm `based prototype - *done*
2.   Add short and Long term `memory` to this prototype - *done*
3.   Add a `custom_tool` to this prototype



In [27]:
embedder = {
    "provider": "google",
    "config": {
    "model": 'models/text-embedding-004',
    "api_key": GEMINI_API_KEY,
    }
}

In [28]:
from crewai import Agent, Crew, Task, Process, LLM # llm is form litellm of crewai
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from textwrap import dedent
from crewai import Agent, Crew, Task, Process, LLM # llm is form litellm of crewai
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

from crewai.memory import ShortTermMemory, LongTermMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage
from typing import List, Optional

from textwrap import dedent

# Create a knowledge source
content = dedent("""
  M.Sufyan Ahmed is a 22 years Electronics Engr lives in Karachi Pakistan, that is working in the domain of semiconductor chip design industry /
  He is an AI enthusiast and fitness freak as well. /
  He also works as an AI/ML & GenAI Engr in part time /
""")

string_source = StringKnowledgeSource(
    content=content,
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
gemini_llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=os.environ.get("GEMINI_API_KEY"), # Assuming GEMINI_API_KEY is set as an environment variable
    temperature=0,
)

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
    embedder=embedder,
)

task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

string_source = StringKnowledgeSource(
    content=content,
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
gemini_llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0,
)

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": GEMINI_API_KEY,
        }
    }
)

task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

In [29]:
crew = Crew(
    agents=[agent],
    tasks=[task],
    memory=True,
   # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./sufy_crew/long_term_memory_storage.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=embedder,
                type="short_term",
                path="./sufy_crew/"
            )
        ),
    verbose=True, #it is used to print outputs to the console
    process=Process.sequential,
    knowledge_sources=[string_source],
    embedder=embedder,
)
def my_knowldge():
    result = crew.kickoff(inputs={"question": "What city does Sufyan live?"})
    print(result)

In [30]:
print(my_knowldge())

# Agent: About User
## Task: Answer the following questions about the user: What city does Sufyan live?


# Agent: About User
## Final Answer: 
Karachi, Pakistan


Karachi, Pakistan
None
